In [ ]:
import notion_client
import sys
sys.path.append("../helper_functions")
import notionapi_utils as n

In [ ]:
# https://oplabs.notion.site/26d856d5ad7c4fda919c62e839cf6051?v=4e38174b2e994129b51dcfa127965aa4

op_summerdb_id = '26d856d5ad7c4fda919c62e839cf6051'

n.read_notion_database(op_summerdb_id)